In [1]:
import pandas as pd
jeopardy = pd.read_csv("jeopardy.csv")
jeopardy.head()

Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY   $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...   $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE   $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES   $200   

                                            Question      Answer  
0  For the last 8 years of his life, Galileo was ...  Copernicus  
1  No. 2: 1912 Olympian; football star at Carlisl...  Jim Thorpe  
2  The city of Yuma in this state has a record av...     Arizona  
3  In 1963, live on "The Art Linkletter Show", th...  McDonald's  
4  Signer of the Dec. of Indep., framer of the Co...  John Adams

In [2]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [3]:
cols = jeopardy.columns.tolist()
for i, each in enumerate(cols):
    t = each
    t = t.strip()
    cols[i] = t
jeopardy.columns = cols

In [4]:
import re
def norm(string):
    string = string.lower()
    string = re.sub("[^a-z0-9\s]", "",string)
    return string
    
jeopardy["clean_question"] = jeopardy["Question"].apply(norm)
jeopardy["clean_answer"] = jeopardy["Answer"].apply(norm)    

In [5]:
def normalize_values(text):
    text = re.sub("[^0-9]", "", text)
    try:
        text = int(text)
    except Exception:
        text = 0
    return text

jeopardy["clean_value"] = jeopardy["Value"].apply(normalize_values)

In [6]:
jeopardy["Air Date"] = pd.to_datetime(jeopardy["Air Date"])

In [7]:
jeopardy.dtypes

Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object

In [8]:
def qa(row):
    split_answer = row["clean_answer"].split(" ")
    split_question = row["clean_question"].split(" ")
    match_count = 0
    try:
        split_answer.remove("the")
    except:
        pass
    if len(split_answer) == 0:
        return 0
    for each in split_answer:
        if each in split_question:
            match_count += 1
    return match_count/len(split_answer)

jeopardy["answer_in_question"] = jeopardy.apply(qa, axis = 1)

In [9]:
jeopardy["answer_in_question"].mean()

0.060493257069335872

Meaning...??

In [10]:
question_overlap = []
terms_used = set()

for i, r in jeopardy.iterrows():
    split_question = r["clean_question"].split(" ")
    split_question = [w for w in split_question if len(w)>5]
    
    match_count = 0
    for each in split_question:
        if each in terms_used:
            match_count += 1
        terms_used.add(each)
    if len(split_question)>0:
        match_count /= len(split_question)
    question_overlap.append(match_count)
        
jeopardy["question_overlap"] = question_overlap
print (jeopardy["question_overlap"].mean())

0.692596005734


Thoughts..??

In [11]:
def lvq(row):
    if row["clean_value"] > 800:
        return 1
    else:
        return 0
    
jeopardy["high_value"] = jeopardy.apply(lvq, axis = 1)

In [15]:
def counts(word):
    low_count = 0
    high_count = 0
    
    for i, r in jeopardy.iterrows():
        split_q = r["clean_question"].split(" ")
        if word in split_q:
            if r["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

observed_expected = []
terms_used = list(terms_used)
comparison_terms = terms_used[:5]

for each in comparison_terms:
    observed_expected.append(counts(each))
observed_expected

[(0, 1), (8, 32), (3, 4), (0, 3), (2, 7)]

In [25]:
high_value_count = jeopardy[jeopardy["high_value"] == 1].shape[0]
low_value_count = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []
from scipy.stats import chisquare

for each in observed_expected:
    total = sum(each)
    total_prop = total/jeopardy.shape[0]
    exphc = total_prop * high_value_count
    explc = total_prop * low_value_count
    cs, pvalue = chisquare((each[0],each[1]), (exphc,explc))
    chi_squared.append((cs,pvalue))

Thoughts on significance...??

Further steps...?